# After 74885

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
#matplotlib figure size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
from mri_project.custom_imports import *
from mri_project.utility import dfe
__spec__ = 'Mardas'

In [57]:
root = "../data/images_with_predictions_v04"
files = glob.glob(root+"/*data")
file = np.random.choice(files)

In [58]:
def get_write_dir(id_):
    id_ = id_.replace('../data/mri_analysis', '/mri_analysis_behnam')\
        .replace(" Set#", '/Set').replace(' ', '__').replace(':', "_")\
        .replace('..', 'blahblah').replace('.', '_').replace('blahblah', '..')
    d, f, e = dfe(id_)
    return root+d, f, e

In [59]:
import scipy.ndimage
import imageio

In [64]:
from uuid import uuid4
predicted = []
traced = []
def rescale(x):
    return np.uint8((x-x.min())/(x.max()-x.min())*255)
def map_fun(file, index=-1):
    # print(index, file)
    # if index % 40 == 0:
    #     print(f"{index}:{file}")
    data = joblib.load(file)
    # data.predicted = data.predicted_postprocessed
    data.get_traced_contours(89.9)
    data.get_predicted_contours(89.9)
    data.get_contour_areas()
    data.get_contour_centers() 
    dic = {
        'raw': data.raw_image,
        'traced': data.traced_image,
        'predicted': data.predicted,
        'traced_lever_arms_90': data.traced_lever_arm_images[89.9]+150,
        'predicted_lever_arms_90': data.predicted_lever_arm_images[89.9]+150,
    }
    dr, f, e = get_write_dir(data.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    for k, v in dic.items():
        # plt.imshow(v)
        # plt.axis('off')
        # plt.savefig(f"{dr}/{k}.png", dpi=150, bbox_inches=0)
        f_name = f"{dr}/{k}_{f}.png"
        # print(f_name)
        v2 = rescale(v)
        print(v2.shape)
        imageio.imsave(f_name, v2)
    plt.imshow(v2)
    return data.id, {'traced': data.traced_features, 'predicted': data.predicted_features}

In [65]:
# for index, file in enumerate(files):
#     if index % 40 == 0:
#         print(index, file)
#     map_fun(file)
#     break

In [66]:
import dask.bag as db

In [ ]:
res = db.from_sequence(files).map(map_fun).compute()

In [45]:
from flatten_dict import flatten

In [46]:
res_fl = flatten(dict(res))

In [47]:
df = pd.DataFrame(res_fl)
df = df.transpose().reset_index()
df['value'] = df[0].combine(df[1], lambda x, y: x if x == y else (x, y))
df = df.drop([0, 1], axis=1)
df.columns = ['id', 'type', 'metric', 'muscle', 'value']

In [48]:
df

,id,type,metric,muscle,value
0,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,1,0.00578261
1,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,4,4.13338
2,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,5,2.20564
3,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,6,2.36826
4,../data/mri_analysis/2.Multigroup Set#2/Contro...,traced,lever_arm_89.9,7,2.38706
...,...,...,...,...,...
50467,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,7,"(609.0930232558139, 883.6478405315614)"
50468,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,8,"(806.2186046511628, 733.6883720930233)"
50469,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,9,"(604.294776119403, 751.5111940298508)"
50470,../data/mri_analysis/2.Multigroup Set#2/Weight...,predicted,center,10,"(1056.7898640296662, 536.3152039555006)"


In [49]:
df.to_csv(root+"/all_features_3bd0c.csv", index=False)

In [50]:
df_piv = df.pivot(index=['id', 'muscle'], columns=['type', 'metric'])['value'].reset_index()
df_piv.to_csv(root+"all_features_pivot_3bd0c.csv", index=False)

In [51]:
df_piv

type                                                   id muscle  \
metric                                                             
0       ../data/mri_analysis/1.Multigroup Set#1_DONE/C...      1   
1       ../data/mri_analysis/1.Multigroup Set#1_DONE/C...      2   
2       ../data/mri_analysis/1.Multigroup Set#1_DONE/C...      3   
3       ../data/mri_analysis/1.Multigroup Set#1_DONE/C...      4   
4       ../data/mri_analysis/1.Multigroup Set#1_DONE/C...      5   
...                                                   ...    ...   
8475    ../data/mri_analysis/4.Contiguous Set#2/Weight...      7   
8476    ../data/mri_analysis/4.Contiguous Set#2/Weight...      8   
8477    ../data/mri_analysis/4.Contiguous Set#2/Weight...      9   
8478    ../data/mri_analysis/4.Contiguous Set#2/Weight...     10   
8479    ../data/mri_analysis/4.Contiguous Set#2/Weight...     11   

type           traced                                                    \
metric lever_arm_89.9     area                                   center   
0           0.0185627   12.317   (704.6863753213368, 701.8071979434447)   
1             3.14821  5.47881   (587.8949579831933, 737.7142857142857)   
2             3.33941  5.70134   (827.2916666666666, 739.4541666666667)   
3             5.45915  2.12653  (502.75132275132273, 812.2169312169312)   
4             4.06823  26.8763          (554.2135416666666, 890.984375)   
...               ...      ...                                      ...   
8475          4.40486  18.3015  (769.8281505728314, 1025.4926350245498)   
8476          3.17714  5.50711  (1162.9712460063897, 817.3450479233227)   
8477          3.14522  4.50121    (834.8545034642033, 825.635103926097)   
8478          10.2864  17.4392  (1531.6973451327433, 675.7619469026548)   
8479          11.7649  16.6216   (387.2413101604278, 681.4110962566845)   

type        predicted                                                    
metric lever_arm_89.9     area                                   center  
0           0.0233926  14.5728   (713.8634920634921, 706.1746031746031)  
1             5.86113  5.66168   (929.4948979591836, 831.4897959183673)  
2             6.17987  5.43731   (485.1447963800905, 810.5610859728507)  
3             4.06626   8.4356   (862.5828877005348, 441.0534759358289)  
4             4.79447  8.80135   (535.6304985337243, 446.3284457478006)  
...               ...      ...                                      ...  
8475          3.13815  17.4966  (829.5743707093822, 1019.6109839816934)  
8476          3.24576  6.30511   (1160.5714285714287, 831.478021978022)  
8477          2.72554  5.51101   (850.6019184652279, 798.1798561151079)  
8478          9.74428  19.5004  (1497.5665188470066, 671.9944567627495)  
8479          11.1745  27.9463   (411.9781659388646, 729.4872114784779)  

[8480 rows x 8 columns]